In [1]:
# check also https://www.programmableweb.com/news/how-to-access-any-restful-api-using-r-language/how-to/2017/07/21?page=2

In [2]:
require('httr')
require('jsonlite')
r = GET("https://www.googleapis.com/books/v1/volumes?q=python")
print(r)

Loading required package: httr

Loading required package: jsonlite



Response [https://www.googleapis.com/books/v1/volumes?q=python]
  Date: 2020-03-28 21:38
  Status: 200
  Content-Type: application/json; charset=UTF-8
  Size: 34.2 kB
{
 "kind": "books#volumes",
 "totalItems": 462,
 "items": [
  {
   "kind": "books#volume",
   "id": "h56ansk4SyQC",
   "etag": "eaypCXf0zmk",
   "selfLink": "https://www.googleapis.com/books/v1/volumes/h56ansk4SyQC",
   "volumeInfo": {
...


In [3]:
r_text <- content(r, "text")
data_json <- jsonlite::fromJSON(r_text, flatten = TRUE)
d = as.data.frame(data_json)

In [4]:
require('httr')
require('jsonlite')
require(glue)
library(data.table)

i = 0
j = 1

alldata = list()
while (TRUE) {
    r = GET(glue("https://www.googleapis.com/books/v1/volumes?q=python&maxResults=40&startIndex={i}"))
    r_text <- content(r, "text")
    data_json <- jsonlite::fromJSON(r_text, flatten = TRUE)
    if (length(data_json$items)==0) {break}
    alldata[[j]] = as.data.frame(data_json)
    i = i + 40
    j = j + 1} 

d = rbindlist(alldata, fill=TRUE)

Loading required package: glue



In [ ]:
# Using twitter to get tweets and user info
lookup_users("vanatteveldt")  %>% select("user_id", "screen_name", "location")
tweets = search_tweets("#covid", n = 100, include_rts = FALSE) 
head(tweets)

In [ ]:
# Getting location data
library(tmaptools)
geocode_OSM(c("Amsterdam", "New York"))

In [ ]:
reply_users = setdiff(tweets$reply_to_screen_name, tweets$screen_name) %>% na.omit()
reply_users = lookup_users(reply_users) %>% select(screen_name, location) 
# Combine with existing users, filter blank locations and duplicate names, and clean location names
users = tweets %>% select(screen_name, location) %>%
  bind_rows(reply_users) %>% 
  unique() %>%
  filter(location != "") %>% 
  mutate(location = str_replace_all(location, "[^\\p{L} ,/-]+", " "))

# Use Open Street Map API to "geocode" the locations and merge back with users
locations = geocode_OSM(unique(users$location)) %>% 
  select(location=query, lat, long=lon)
users = users %>% left_join(locations)
head(users)